### USA

+ `admin_name1` = `region`
+ `place_name` = `city`

In [2]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

**Crunchbase Dataframe**

In [3]:
df = pd.read_csv('input/foodtech.csv')
df = df[df['country_code'] == 'USA']
df = df[['uuid','country_code', 'state_code', 'region', 'city', 'address', 'postal_code']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62446 entries, 0 to 163819
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uuid          62446 non-null  object
 1   country_code  62446 non-null  object
 2   state_code    62446 non-null  object
 3   region        62446 non-null  object
 4   city          62446 non-null  object
 5   address       44895 non-null  object
 6   postal_code   45652 non-null  object
dtypes: object(7)
memory usage: 3.8+ MB


*Reformatting*

In [60]:
df['city'] = df['city'].apply(lambda x: 'Saint ' + x[3:] if x.startswith('St ') else x)
df['city'] = df['city'].apply(lambda x: 'Saint ' + x[4:] if x.startswith('St. ') else x)

**Postal Codes**

`admin_name1`

In [61]:
codes = pd.read_json('input/geonames.json')
codes = codes[codes['country_code'] == 'US']

In [62]:
codes['postal_code'] = codes['postal_code'].astype(str).str.zfill(5)

In [63]:
codes = codes[['postal_code', 'place_name', 'admin_name1', 'admin_code1']]
codes.drop_duplicates(subset=['place_name', 'admin_name1', 'admin_code1'], inplace=True)

*Merging*

In [64]:
merged_df = pd.merge(df, codes, left_on=['city', 'region'], right_on=['place_name', 'admin_name1'], how='left')

In [65]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62446 entries, 0 to 62445
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           62446 non-null  object
 1   country_code   62446 non-null  object
 2   state_code     62446 non-null  object
 3   region         62446 non-null  object
 4   city           62446 non-null  object
 5   address        44895 non-null  object
 6   postal_code_x  45652 non-null  object
 7   postal_code_y  61352 non-null  object
 8   place_name     61352 non-null  object
 9   admin_name1    61352 non-null  object
 10  admin_code1    61352 non-null  object
dtypes: object(11)
memory usage: 5.7+ MB


*Checking matches*

In [66]:
matches = merged_df[['postal_code_x', 'postal_code_y']]
matches['match'] = matches['postal_code_x'] == matches['postal_code_y']
matches['match'].value_counts()

False    46835
True     15611
Name: match, dtype: int64

In [67]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62446 entries, 0 to 62445
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   postal_code_x  45652 non-null  object
 1   postal_code_y  61352 non-null  object
 2   match          62446 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 1.5+ MB


Differences can be attributed to 
+ NaN in crunchbase data
+ Same city, different neighbourhood: 75014 vs 75009

*Dealing with NaNs*

+ NaNs attributed to cities not present in external database. Why? idk

In [68]:
nan_df = merged_df[merged_df['postal_code_y'].isna()]

In [69]:
nan_df[['city','postal_code_x', 'postal_code_y']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 256 to 62378
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   city           1094 non-null   object
 1   postal_code_x  826 non-null    object
 2   postal_code_y  0 non-null      object
dtypes: object(3)
memory usage: 34.2+ KB


**Filling NaNs with Existing Postal Codes**

In [70]:
merged_df['postal_code_y'].fillna(merged_df['postal_code_x'], inplace=True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62446 entries, 0 to 62445
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           62446 non-null  object
 1   country_code   62446 non-null  object
 2   state_code     62446 non-null  object
 3   region         62446 non-null  object
 4   city           62446 non-null  object
 5   address        44895 non-null  object
 6   postal_code_x  45652 non-null  object
 7   postal_code_y  62178 non-null  object
 8   place_name     61352 non-null  object
 9   admin_name1    61352 non-null  object
 10  admin_code1    61352 non-null  object
dtypes: object(11)
memory usage: 5.7+ MB


*Across rows: df*

In [71]:
merged_df = merged_df[merged_df.columns[:-3]]

In [72]:
codes_df = df[df['postal_code'].notna()]

codes_df = codes_df[['region', 'city', 'postal_code']]
codes_df.drop_duplicates(subset=['region', 'city'], inplace=True)

In [73]:
merged_df = pd.merge(merged_df, codes_df, left_on=['city', 'region'], right_on=['city', 'region'], how='left')
merged_df['postal_code_y'].fillna(merged_df['postal_code'], inplace=True)

In [74]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62446 entries, 0 to 62445
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uuid           62446 non-null  object
 1   country_code   62446 non-null  object
 2   state_code     62446 non-null  object
 3   region         62446 non-null  object
 4   city           62446 non-null  object
 5   address        44895 non-null  object
 6   postal_code_x  45652 non-null  object
 7   postal_code_y  62420 non-null  object
 8   postal_code    61338 non-null  object
dtypes: object(9)
memory usage: 4.8+ MB


*Checking NaNs*

In [76]:
nan_df = merged_df[merged_df['postal_code_y'].isna()]
nan_df['region'].value_counts()

Alabama         5
New Jersey      3
Louisiana       2
Wisconsin       2
New York        2
Florida         2
Pennsylvania    1
Arizona         1
Alaska          1
Tennessee       1
Ohio            1
Michigan        1
Oregon          1
Oklahoma        1
Indiana         1
Mississippi     1
Name: region, dtype: int64

*Save to csv*

In [77]:
merged_df = merged_df.rename(columns={'postal_code_x': 'pc_crunchbase', 
                                      'postal_code_y': 'pc_filled'})

merged_df = merged_df[['uuid', 'pc_crunchbase', 'pc_filled']]


In [78]:
merged_df.to_csv('processed/USA_processed.csv')